# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Here we have two possible values: students that need intervention and students who don't. This means that we are dealing with discrete values, therefore, this is a classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [36]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [37]:
# TODO: Calculate number of students
n_students = np.shape(student_data)[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 # last column is the target

# TODO: Calculate passing students
n_passed = np.shape(np.where(student_data.passed == "yes"))[1]

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = 100.00 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [38]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [39]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [40]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=10, stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

- **DECISION TREES**

    *General Applications*
    
    This approach is used in several industries to compare different scenarios and make the best decision. For instance, a company can use decision trees to decide whether to sell new products in the current market, or to sell the current products in a new market. It is also used in the field of object recognition, for instance to identify a face as male or female.
    
    *Strengths*
    - Easy to understand and to implement. We can visualize the tree.
    - It allows to indentify the most significat feature of a dataset.
    - It can handle datasets where some data is missed.

 *Weaknesses*
    - Its accuracy can be misleading when dealing with unbalanced dataset.
    - It's prone to overfitting when the depth of the tree and the minimum number of elements to split a node are not set properly.
    - Even though it can be used with continuos variables, it accuracy it is reduced when categorize variables in different categories.

 *Reasons for choosing*
    - In the current dataset there are 30 features, which is a considerable number. With decision trees it is possible to identify which of the features provide more useful information to make a proper prediction.
    - The dataset has features with discrete values, for instance internet (yes or no), sex (male or female), so the nodes can be split more easily. There are some other features that can take more values like absences (greater or equal than 0) but they are discrete too, if necessary those values can be grouped within given ranges (0 - 5, 6- 10, 11 - 15).


- **SUPPORT VECTOR MACHINES**

    *General Applications*
    
    It is used in for classification images, since it performs well with multi-category datasets; for the same reason it's used to recognize hand-written characters.
    
    *Strengths*
    - It can handle well a high number of features.
    - It can deal with unbalanced dataset by weighing the classes.
    - Versatile, since we can use different kernel functions for decision function.
    
 *Weaknesses*
    - It's a black box. Unlike Decision Trees, where it is easy to understand why an element has fallen into a category, this is not really possible with SVM.
    - Its training time is high, so it does not perfom well in large datasets.
    
 *Reasons for choosing*
    - Our dataset is not that big (just a few hundreds of rows), so the time to train SVM won't be really high.
    - We have a considerable number of features (30) that is a high dimensional space, something easy to handle with SVM


- **KNN**

    *General Applications*
    
   It can be used to predict when an employee is likely to quit, to predict the stock price of a new product. It is mostly used for classification problems.
    
    *Strengths*
    - It's an easy to understand.
    - It's not a black box, we can know why an element has been categorized as such.
    - We can increase the accuracy of the prediction with more data. This is a tradeoff since the more data the higher the accuracy, but also the higher the computational resources needed.
    
 *Weaknesses*
    - It can be computationally expensive when classifying a large number of instances.
    - The presence of outliers can affect the prediction significantly.
    - It uses a lot of memory to store all the instances
    
 *Reasons for choosing*
 
   Our dataset has just a few hundreds of rows, so it won't be really expensive to use KNN. At first, it looks this is the weakest one, but it's a good opportunity to compare its performance and accuracy against two more robust approaches. 

Based on the current knowledge, the expectation is that SVM gives better results than Decision trees, which in turn should perform better than KNN.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [41]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [42]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import svm
from sklearn import tree
from sklearn import neighbors

# TODO: Initialize the three models
clf_A = svm.SVC(random_state=10)
clf_B = tree.DecisionTreeClassifier(random_state=10)
clf_C = neighbors.KNeighborsClassifier()

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [1, 2, 3]:
        train_predict(clf, X_train[:100*n], y_train[:100*n], X_test, y_test)
        print "\n"



SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0018 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8375.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7975.


Training a SVC using a training set size of 200. . .
Trained model in 0.0044 seconds
Made predictions in 0.0033 seconds.
F1 score for training set: 0.8750.
Made predictions in 0.0017 seconds.
F1 score for test set: 0.7895.


Training a SVC using a training set size of 300. . .
Trained model in 0.0086 seconds
Made predictions in 0.0064 seconds.
F1 score for training set: 0.8602.
Made predictions in 0.0021 seconds.
F1 score for test set: 0.8212.



DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7188.


Training a DecisionTreeClassifier using a trai

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0011           |          0.0010        |      0.8375      |      0.7975     |
| 200               |        0.0033           |          0.0017        |      0.8750      |      0.7895     |
| 300               |        0.0064           |          0.0021        |      0.8602      |      0.8212     |

** Classifer 2 - Decision tree**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0002           |          0.0002        |      1.0000      |      0.7188     |
| 200               |        0.0002           |          0.0002        |      1.0000      |      0.7407     |
| 300               |        0.0003           |          0.0002        |      1.0000      |      0.7883     |

** Classifer 3 - KNN**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0015           |          0.0014        |      0.7733      |      0.7862     |
| 200               |        0.0040           |          0.0024        |      0.8322      |      0.8028     |
| 300               |        0.0082           |          0.0030        |      0.8676      |      0.8088     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Based on the prediction time and F1 score, we see the following: 

Decision tree has lower prediction times because it basically process the data with simple conditions. SVM takes more time because it has a higher complexity due to the high dimensional spaces it uses to categorize the data. As expected, KNN has the highest prediction times since it is a lazy predictor. 

In regards to F1 score, in the test stage, we see that SVM has a better behaviour. This is expected since it did its heaviest work in the training stage to find the hyperplanes that best categorizes the data. However, it is worth to mention that the best result in all the predictions belongs to KNN when working with a training set of 300 elements. This makes sense since there are more data to work with; KNN basically averages the values of the nearest neighbors, hence, the more the data the better the prediction.

In the test stage we see that Decision tree as a perfect F1 Score. This is bad news because the model is overfitting the training data. However, it is not unexpected because we used the default values for the classifiers. For this specific classifier, we have that the default values for the parameters min_samples_leaf and min_samples_split are 1 and 2 respectively. This means that the algorithm is splitting the dataset until we have just one element in every leaf node. This clearly does not generalizes well.

In conclusion, given the above results, the current settings and the posterior analysis, and keeping in mind that the chosen size for the training set is 300, the best model is KNN because, despite its higher predictions times, it keeps things simple with a good score.



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

KNN works like Democracy, by voting. Let's say there is an element we want to categorize. We know some information about that element. We also have the same information about other elements and the category they belong to. With this information we chose the K nearest neighbors to our element and let them vote. The nearest neighbors to the element are chosen based on the distance to it. This distance is not a necessarily a measurement of space, it is how far an object is from another based on a given characteristic. For instance, if the feature is the age, a 35 years-old individual is closer to a 29 years-old one than to a 25 years-old.

Now, that we have the K nearest neighbors, it's time to vote. The easiest way to define the category of our element is by simple majority. For instance we have 9 neighbors, 4 of them belong to category A and the other 5 to category B, therefore our element is categorized as a B one. It is also possible to assign weights to the neighbors. In the previous example, let's say that the A neighbors are much closer to the element that the B ones. A ones have more influence than B ones, their votes weigh more, therefore, even though there are fewer A neighbors, our element is categorized as A.



### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [79]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'n_neighbors': [1, 20, 30], 'leaf_size': [10, 40]}

# TODO: Initialize the classifier
clf = neighbors.KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better=True, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0136 seconds.
Tuned model has a training F1 score of 0.8118.
Made predictions in 0.0049 seconds.
Tuned model has a testing F1 score of 0.7919.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
F1 score for training: 0.8118
F1 score for testing: 0.7919

In the case of training, the score of the tuned model is lower, this clearly happens because the number of neighbors chosen is higher in the tuned one. Originally, one may think that this value is not good because we use more elements to do the classification, therefore the accuracy should be higher (F1 score should be higher with tuned model). However, given that the difference is not that big (0.8676 - 0.8118 = 0.0558, that is 6% lower).

In the case of testing, the new score is also lower, the difference is much lower than in the training stage (0.8088 - 0.7919 = 0.0038, that is 2% lower).

It seems that modifying the hyperparameters, we don't really get better results. Given the small difference between the default and the tuned scores, we can't say that the new model really behaves worse either. We have the option to increase the size of the dataset to get better results, but as mentioned previously, there is tradeoff between accuracy and computational expensiveness for this particular algorithm.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.